In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

## Determining the rock type boundaries

In [ ]:
import pandas as pd
from quick_pp.rock_type import plot_ward, plot_lorenz

core_data = pd.read_csv(r'data\01_raw\HW_core_data_all.csv')
core_data['CPORE'] = core_data['Phi (frac.)']
core_data['CPERM'] = core_data['K mD']

clean_core_data = core_data.drop_duplicates(subset=['Sample', 'Class', 'Phi (frac.)', 'K mD'], keep='last')
plot_ward(clean_core_data['CPORE'], clean_core_data['CPERM'])
plot_lorenz(core_data['CPORE'], core_data['CPERM'])

## Plotting the Rock Types

In [ ]:
from quick_pp.rock_type import plot_ward, plot_fzi, plot_rfn, plot_lorenz, calc_fzi, rock_typing

log_fzi_cut_offs = [-.88, -.479, -.179, .146, .571, 1.35]
fzi_cut_offs = [round(10**(i), 3) for i in log_fzi_cut_offs]

all_data = project.get_all_data()
all_data['CPERM'] = all_data.CORE_PERM
all_data['CPORE'] = all_data.CORE_POR / 100

# Estimate rock types
fzi = calc_fzi(all_data['CPERM'], all_data['CPORE'])
all_data['FZI'] = fzi
rock_flag = rock_typing(fzi, higher_is_better=True, cut_offs=fzi_cut_offs)
all_data['ROCK_FLAG'] = rock_flag

plot_rfn(all_data['CPORE'], all_data['CPERM'], rock_type=rock_flag)
plot_fzi(all_data['CPORE'], all_data['CPERM'], fzi=fzi_cut_offs, rock_type=rock_flag)

print(fzi_cut_offs)
print(pd.Series(rock_flag).value_counts())

## Building Rock Type and FZI model to propagate to non-cored intervals

In [ ]:
import pickle
from quick_pp.rock_type import train_rock_type, train_fzi

train_data = all_data.dropna(subset=['GR', 'NPHI', 'RHOB', 'ROCK_FLAG'])
rt_model = train_rock_type(train_data[['GR', 'NPHI', 'RHOB']], train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\rt_model.qppm', 'wb') as file:
    pickle.dump(rt_model, file)


train_data = all_data.dropna(subset=['GR', 'NPHI', 'RHOB', 'FZI'])
fzi_model = train_fzi(train_data[['GR', 'NPHI', 'RHOB']], train_data['FZI'], train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'wb') as file:
    pickle.dump(fzi_model, file)

## Predict PERM and plot the results

In [ ]:
from quick_pp.plotter import plotly_log
from quick_pp.rock_type import calc_fzi_perm

# Plot individual results
well_data = all_data[all_data.WELL_NAME == 'HW-29'].copy()
well_data['ROCK_FLAG'] = rt_model.predict(well_data[['GR', 'NPHI', 'RHOB']])
well_data['FZI'] = fzi_model.predict(well_data[['GR', 'NPHI', 'RHOB']])
well_data['PERM'] = well_data.apply(lambda row: calc_fzi_perm(row['FZI'], row['PHIE']), axis=1)
fig = plotly_log(well_data, '')
fig.show(config=dict(scrollZoom=True))